In [1]:
# Import SQLAlchemy `automap` and other dependencies here
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
from sqlalchemy import MetaData
from sqlalchemy import Table
from sqlalchemy import desc
from sqlalchemy import func

from flask import Flask, render_template, jsonify

import pandas as pd
import numpy as np

In [2]:
app = Flask(__name__)

In [3]:
engine = create_engine("sqlite:///healthcare.db")
print("Connected to DB")

# reflect an existing database into a new model
Base = automap_base()                                                                                                                                                                                                                                                                                                           
# reflect the tables
Base.prepare(engine, reflect=True)
print("Reflected tables")

# Create our session (link) from Python to the DB
session = Session(bind=engine)

print(Base.classes.keys())

Connected to DB
Reflected tables
['ALL_STATES_GDP', 'ALL_STATES_PER_CAPITA_GDP', 'COUNTRIES_HEALTHCARE_SPENDING', 'MEDICAID_PER_CAPITA', 'MEDICAID_POPULATION', 'MEDICAID_TOTAL_SPENDING', 'MEDICARE_PER_CAPITA', 'MEDICARE_POPULATION', 'MEDICARE_TOTAL_SPENDING', 'PHI_PER_CAPITA', 'PHI_POPULATION', 'PHI_TOTAL_SPENDING', 'US_PER_CAPITA', 'US_POPULATION', 'US_TOTAL_SPENDING']


In [4]:
# Save reference to the table
ALL_STATES_GDP = Base.classes.ALL_STATES_GDP
US_TOTAL_SPENDING = Base.classes.US_TOTAL_SPENDING

In [5]:
state = 'Alabama'
s1 = session.query(ALL_STATES_GDP).filter(ALL_STATES_GDP.State_Name == state).first()
state_info = vars(s1)
print(state_info)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001D30CD680B8>, '2010': 174710, '2008': 172646, '2006': 164468, '2004': 146989, '2002': 127873, '2001': 122822, 'State_Name': 'Alabama', '2013': 190319, '2011': 180665, '2009': 168315, '2007': 169923, '2005': 156750, '2003': 134388, 'ComponentName': 'Gross domestic product (GDP) by state (millions of current dollars)', '2014': 193995, '2012': 185878}


In [6]:
state = 'Alabama'
gdp_query = "SELECT * FROM ALL_STATES_GDP where State_Name = '" + state +"'"
results = engine.execute(gdp_query).fetchall()
gdp_df = pd.DataFrame(results, columns=['State_Name', 'Component', '2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014'])
gdp_df.set_index('State_Name', inplace=True, )
gdp_df = gdp_df[['2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014']]
gdp_df.head()

,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
State_Name,,,,,,,,,,,,,,
Alabama,122822,127873,134388,146989,156750,164468,169923,172646,168315,174710,180665,185878,190319,193995


In [7]:
nhs_query = "SELECT * FROM US_TOTAL_SPENDING where State_Name = '" + state +"'"
results = engine.execute(nhs_query).fetchall()
nhs_df = pd.DataFrame(results, columns=['Item', 'State_Name', '2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','annual_growth_perentage'])
nhs_df.set_index('State_Name', inplace=True, )
nhs_df = nhs_df[['2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014']]
nhs_df.head()

,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
State_Name,,,,,,,,,,,,,,
Alabama,18619,20209,22491,23797,25338,26638,27700,28765,30095,30728,31398,32848,33788,35263


In [8]:
tmp_gdp_df = gdp_df.iloc[0]
tmp_gdp_df = tmp_gdp_df.pct_change()
tmp_gdp_df = list(tmp_gdp_df[1:len(tmp_gdp_df)])
tmp_gdp_df

[0.041124554232955024,
 0.05094898844947715,
 0.09376581242372839,
 0.06640632972535365,
 0.04923763955342908,
 0.033167546270399084,
 0.01602490539832746,
 -0.02508601415613454,
 0.03799423699610838,
 0.03408505523438832,
 0.028854509727949562,
 0.02389201519276085,
 0.01931493965394937]

In [9]:
tmp_nhs_df = nhs_df.iloc[0]
tmp_nhs_df = tmp_nhs_df.pct_change()
tmp_nhs_df = list(tmp_nhs_df[1:len(tmp_nhs_df)])
tmp_nhs_df

[0.0853966378430635,
 0.11291998614478693,
 0.0580676715130497,
 0.06475606168844816,
 0.05130633830610143,
 0.039867857947293306,
 0.03844765342960299,
 0.04623674604554151,
 0.02103339425153683,
 0.0218042176516533,
 0.04618128543219324,
 0.028616658548465734,
 0.04365455191192136]

In [10]:
COUNTRIES_HEALTHCARE_SPENDING = Base.classes.US_TOTAL_SPENDING

In [11]:
country_list = "('Canada','United Kingdom','United States','Switzerland','Sweden','Australia')"
query = "SELECT * FROM COUNTRIES_HEALTHCARE_SPENDING where Country IN " + country_list
results = engine.execute(query).fetchall()
country_df = pd.DataFrame(results, columns=['Country', '2000', '2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016'])
country_df.set_index('Country', inplace=True )
country_df

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Country,,,,,,,,,,,,,,,,,
Australia,"2,767.50","2,872.20","2,999.30","3,090.50","3,240.20","3,242.90","3,323.30","3,415.30","3,494.50","3,629.70","3,607.50","3,738.30","3,830.30","3,899.60","4,057.80","4,278.30","4,387.70"
Canada,"3,054.90","3,216.50","3,365.90","3,454.90","3,549.70","3,615.70","3,731.00","3,808.60","3,873.50","4,151.90","4,227.80","4,177.70","4,202.10","4,211.60","4,218.20","4,411.50","4,481.10"
Sweden,"2,650.80","2,910.20","3,081.70","3,181.80","3,227.10","3,311.20","3,398.60","3,451.10","3,506.10","3,546.50","3,536.10","4,533.90","4,596.50","4,682.90","4,774.80","4,860.50","4,993.00"
Switzerland,"4,478.10","4,685.10","4,851.00","4,958.90","5,079.60","5,116.90","5,057.50","5,128.40","5,257.30","5,397.10","5,686.30","5,763.90","5,913.30","6,091.90","6,260.80","6,493.80","6,718.10"
United Kingdom,"1,956.80","2,115.30","2,249.10","2,408.30","2,530.50","2,639.80","2,739.80","2,827.60","2,881.80","3,031.80","3,040.70","3,044.90","3,069.70","3,613.10","3,675.30","3,755.60","3,758.80"
United States,"5,635.40","5,934.90","6,341.70","6,695.60","6,923.70","7,091.60","7,268.10","7,447.30","7,549.80","7,772.80","7,931.70","7,980.10","8,095.40","8,157.60","8,404.00","8,748.60","8,984.80"


In [12]:
list(country_df.iloc[0])

['2,767.50',
 '2,872.20',
 '2,999.30',
 '3,090.50',
 '3,240.20',
 '3,242.90',
 '3,323.30',
 '3,415.30',
 '3,494.50',
 '3,629.70',
 '3,607.50',
 '3,738.30',
 '3,830.30',
 '3,899.60',
 '4,057.80',
 '4,278.30',
 '4,387.70']

In [13]:
country_list = country_df.index
country_trend = {}
for i in range(len(country_list)):
    country_trend[country_df.index[i]] = list(country_df.iloc[i]) # list(country_df.loc[country_df.index == country].values)

import json
json.dumps(country_trend)

'{"Australia": ["2,767.50", "2,872.20", "2,999.30", "3,090.50", "3,240.20", "3,242.90", "3,323.30", "3,415.30", "3,494.50", "3,629.70", "3,607.50", "3,738.30", "3,830.30", "3,899.60", "4,057.80", "4,278.30", "4,387.70"], "Canada": ["3,054.90", "3,216.50", "3,365.90", "3,454.90", "3,549.70", "3,615.70", "3,731.00", "3,808.60", "3,873.50", "4,151.90", "4,227.80", "4,177.70", "4,202.10", "4,211.60", "4,218.20", "4,411.50", "4,481.10"], "Sweden": ["2,650.80", "2,910.20", "3,081.70", "3,181.80", "3,227.10", "3,311.20", "3,398.60", "3,451.10", "3,506.10", "3,546.50", "3,536.10", "4,533.90", "4,596.50", "4,682.90", "4,774.80", "4,860.50", "4,993.00"], "Switzerland": ["4,478.10", "4,685.10", "4,851.00", "4,958.90", "5,079.60", "5,116.90", "5,057.50", "5,128.40", "5,257.30", "5,397.10", "5,686.30", "5,763.90", "5,913.30", "6,091.90", "6,260.80", "6,493.80", "6,718.10"], "United Kingdom": ["1,956.80", "2,115.30", "2,249.10", "2,408.30", "2,530.50", "2,639.80", "2,739.80", "2,827.60", "2,881.80",

In [14]:
country_list = country_df.index
country_trend = {}
for country in country_list:
    country_trend[country] = country_df.loc[country_df.index == country].values # list(country_df.loc[country_df.index == country].values)

country_trend

{'Australia': array([['2,767.50', '2,872.20', '2,999.30', '3,090.50', '3,240.20',
         '3,242.90', '3,323.30', '3,415.30', '3,494.50', '3,629.70',
         '3,607.50', '3,738.30', '3,830.30', '3,899.60', '4,057.80',
         '4,278.30', '4,387.70']], dtype=object),
 'Canada': array([['3,054.90', '3,216.50', '3,365.90', '3,454.90', '3,549.70',
         '3,615.70', '3,731.00', '3,808.60', '3,873.50', '4,151.90',
         '4,227.80', '4,177.70', '4,202.10', '4,211.60', '4,218.20',
         '4,411.50', '4,481.10']], dtype=object),
 'Sweden': array([['2,650.80', '2,910.20', '3,081.70', '3,181.80', '3,227.10',
         '3,311.20', '3,398.60', '3,451.10', '3,506.10', '3,546.50',
         '3,536.10', '4,533.90', '4,596.50', '4,682.90', '4,774.80',
         '4,860.50', '4,993.00']], dtype=object),
 'Switzerland': array([['4,478.10', '4,685.10', '4,851.00', '4,958.90', '5,079.60',
         '5,116.90', '5,057.50', '5,128.40', '5,257.30', '5,397.10',
         '5,686.30', '5,763.90', '5,913.30'

In [15]:
country_df.loc[country_df.index == 'Australia']

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Country,,,,,,,,,,,,,,,,,
Australia,"2,767.50","2,872.20","2,999.30","3,090.50","3,240.20","3,242.90","3,323.30","3,415.30","3,494.50","3,629.70","3,607.50","3,738.30","3,830.30","3,899.60","4,057.80","4,278.30","4,387.70"


In [16]:
table = pd.pivot_table(country_df, columns=['Country'], aggfunc='sum')
table

Country,Australia,Canada,Sweden,Switzerland,United Kingdom,United States
2000,"2,767.50","3,054.90","2,650.80","4,478.10","1,956.80","5,635.40"
2001,"2,872.20","3,216.50","2,910.20","4,685.10","2,115.30","5,934.90"
2002,"2,999.30","3,365.90","3,081.70","4,851.00","2,249.10","6,341.70"
2003,"3,090.50","3,454.90","3,181.80","4,958.90","2,408.30","6,695.60"
2004,"3,240.20","3,549.70","3,227.10","5,079.60","2,530.50","6,923.70"
2005,"3,242.90","3,615.70","3,311.20","5,116.90","2,639.80","7,091.60"
2006,"3,323.30","3,731.00","3,398.60","5,057.50","2,739.80","7,268.10"
2007,"3,415.30","3,808.60","3,451.10","5,128.40","2,827.60","7,447.30"
2008,"3,494.50","3,873.50","3,506.10","5,257.30","2,881.80","7,549.80"
2009,"3,629.70","4,151.90","3,546.50","5,397.10","3,031.80","7,772.80"


In [17]:
'''
import matplotlib.pyplot as plt
plt.figure();
tmp_gdp_df.plot();
tmp_nhs_df.plot();
# country_df.plot();
plt.show();
'''

'\nimport matplotlib.pyplot as plt\nplt.figure();\ntmp_gdp_df.plot();\ntmp_nhs_df.plot();\n# country_df.plot();\nplt.show();\n'

In [20]:
year = '2001'
results = engine.execute('SELECT * from US_PER_CAPITA').fetchall()
gdp_df = pd.DataFrame(results, columns=['Item','State_Name', '2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014', 'Avg'])
gdp_df = gdp_df[['State_Name',year]]
gdp_df.set_index('State_Name', inplace=True)
perCapita = gdp_df.to_dict()
#print('Per Capita')
#print(perCapita)
temp_dict = perCapita[year]
#print('Temp Data')
print(temp_dict)
state_per_capita = []
response_dict = {}
for key, value in temp_dict.items():
        # print('key' + key)
        response_dict['zCapita'] = value
        response_dict['State'] = key
        state_per_capita.append(response_dict)
        response_dict = {}
        
state_per_capita

{'Alabama': 4168, 'Alaska': 5169, 'Arizona': 3438, 'Arkansas': 4030, 'California': 3838, 'Colorado': 3992, 'Connecticut': 5560, 'Delaware': 5091, 'District of Columbia': 6663, 'Florida': 4745, 'Georgia': 3806, 'Hawaii': 4104, 'Idaho': 3651, 'Illinois': 4465, 'Indiana': 4344, 'Iowa': 4574, 'Kansas': 4556, 'Kentucky': 4473, 'Louisiana': 4289, 'Maine': 5129, 'Maryland': 4604, 'Massachusetts': 5628, 'Michigan': 4287, 'Minnesota': 4898, 'Mississippi': 4087, 'Missouri': 4467, 'Montana': 4208, 'Nebraska': 4562, 'Nevada': 3717, 'New Hampshire': 4522, 'New Jersey': 4990, 'New Mexico': 3717, 'New York': 5480, 'North Carolina': 4363, 'North Dakota': 4836, 'Ohio': 4661, 'Oklahoma': 4106, 'Oregon': 4155, 'Pennsylvania': 5091, 'Rhode Island': 5188, 'South Carolina': 4238, 'South Dakota': 4379, 'Tennessee': 4494, 'Texas': 3941, 'Utah': 3293, 'Vermont': 4722, 'Virginia': 3857, 'Washington': 4171, 'West Virginia': 4887, 'Wisconsin': 4752, 'Wyoming': 4270}


[{'State': 'Alabama', 'zCapita': 4168},
 {'State': 'Alaska', 'zCapita': 5169},
 {'State': 'Arizona', 'zCapita': 3438},
 {'State': 'Arkansas', 'zCapita': 4030},
 {'State': 'California', 'zCapita': 3838},
 {'State': 'Colorado', 'zCapita': 3992},
 {'State': 'Connecticut', 'zCapita': 5560},
 {'State': 'Delaware', 'zCapita': 5091},
 {'State': 'District of Columbia', 'zCapita': 6663},
 {'State': 'Florida', 'zCapita': 4745},
 {'State': 'Georgia', 'zCapita': 3806},
 {'State': 'Hawaii', 'zCapita': 4104},
 {'State': 'Idaho', 'zCapita': 3651},
 {'State': 'Illinois', 'zCapita': 4465},
 {'State': 'Indiana', 'zCapita': 4344},
 {'State': 'Iowa', 'zCapita': 4574},
 {'State': 'Kansas', 'zCapita': 4556},
 {'State': 'Kentucky', 'zCapita': 4473},
 {'State': 'Louisiana', 'zCapita': 4289},
 {'State': 'Maine', 'zCapita': 5129},
 {'State': 'Maryland', 'zCapita': 4604},
 {'State': 'Massachusetts', 'zCapita': 5628},
 {'State': 'Michigan', 'zCapita': 4287},
 {'State': 'Minnesota', 'zCapita': 4898},
 {'State': 'M